In [1]:
from pybaseball import statcast, statcast_pitcher_spin, statcast_pitcher

data = statcast(start_dt = '2021-04-01', end_dt = '2021-10-04')

data.to_csv('./data/mlb-pitches.csv')
data = pd.read_csv('./data/mlb-pitches.csv')

data.drop(columns = ['Unnamed: 0', 'spin_dir', 'spin_rate_deprecated', 
                     'break_angle_deprecated', 'break_length_deprecated', 
                     'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id'], inplace = True)

# Switch from catcher's perspective to pitcher's perspective
# Catcher's POV: (plate_x, plate_z)
# Pitcher's POV: (plate_-x, plate_z)
data['plate_-x'] = -data['plate_x']
# Switch HB to perspective of pitcher
# Catcher's POV: (pfx_x, pfx_z)
# Pitcher's POV: (pfx_-x, pfx_z)
data['pfx_-x'] = -data['pfx_x']

# HB and VB in feet should be in inches (*12)
data['pfx_x'] = 12 * data['pfx_x']
data['pfx_-x'] = 12 * data['pfx_-x']
data['pfx_z'] = 12 * data['pfx_z']

data.head()

This is a large query, it may take a moment to complete


100%|██████████| 187/187 [00:03<00:00, 49.56it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,plate_-x,pfx_-x
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,field_out,hit_into_play,...,5,0,5,Infield shift,Strategic,148.0,0.000,-0.073,0.69,-8.28
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,...,5,0,5,Infield shift,Strategic,315.0,0.000,-0.027,0.71,9.24
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,...,5,0,5,Infield shift,Standard,328.0,0.000,-0.020,0.04,7.80
3,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,NaN,ball,...,5,0,5,Standard,Standard,330.0,0.000,0.016,-2.10,8.28
4,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,...,5,0,5,Standard,Standard,143.0,0.002,-0.189,0.31,-7.56


In [2]:
data.to_csv('./data/mlb-pitches.csv')